In [47]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
#classifiaction.
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#regression
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

#model selection
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

#evaluation metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score  # for classification
 

In [9]:
files=['2019_01','2019_02','2019_03','2019_04','2019_05','2019_06','2019_07','2019_08','2019_09']

def merge_data(trials_name):
    merged=pd.DataFrame()
    for name in trials_name:
        name1=pd.read_csv(name+'.csv')
        print('load'+name)
        print("shape is", name1.shape)
        print(name1.columns)
        merged=merged.append(name1, ignore_index=True, sort=False)
        print('merge'+name)
        print('shape is', merged.shape)
        print(merged.columns)
    return merged

In [10]:
dataset=merge_data(files)

load2019_01
shape is (583985, 33)
Index(['MONTH', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'ORIGIN_WAC',
       'DEST_AIRPORT_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEST_STATE_NM', 'DEST_WAC', 'WHEELS_ON', 'TAXI_IN',
       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW', 'ARR_DEL15',
       'ARR_DELAY_GROUP', 'ARR_TIME_BLK', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS', 'DISTANCE',
       'DISTANCE_GROUP', 'Unnamed: 32'],
      dtype='object')
merge2019_01
shape is (583985, 33)
Index(['MONTH', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'ORIGIN_WAC',
       'DEST_AIRPORT_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 

merge2019_07
shape is (4293367, 33)
Index(['MONTH', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'ORIGIN_WAC',
       'DEST_AIRPORT_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR', 'DEST_STATE_NM', 'DEST_WAC', 'WHEELS_ON', 'TAXI_IN',
       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW', 'ARR_DEL15',
       'ARR_DELAY_GROUP', 'ARR_TIME_BLK', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS', 'DISTANCE',
       'DISTANCE_GROUP', 'Unnamed: 32'],
      dtype='object')
load2019_08
shape is (658461, 33)
Index(['MONTH', 'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID',
       'ORIGIN_AIRPORT_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_NM', 'ORIGIN_WAC',
       'DEST_AIRPORT_ID', 'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME',
       'DEST_STATE_ABR',

In [11]:
dataset.head(5)

,MONTH,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ORIGIN_WAC,...,ARR_DEL15,ARR_DELAY_GROUP,ARR_TIME_BLK,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,Unnamed: 32
0,1,9E,20363,13487,31650,MSP,"Minneapolis, MN",MN,Minnesota,63,...,0.0,-2.0,1800-1859,121.0,106.0,87.0,1.0,596.0,3,NaN
1,1,9E,20363,13487,31650,MSP,"Minneapolis, MN",MN,Minnesota,63,...,0.0,-2.0,1900-1959,126.0,93.0,76.0,1.0,596.0,3,NaN
2,1,9E,20363,13487,31650,MSP,"Minneapolis, MN",MN,Minnesota,63,...,0.0,-2.0,1900-1959,126.0,118.0,76.0,1.0,596.0,3,NaN
3,1,9E,20363,13487,31650,MSP,"Minneapolis, MN",MN,Minnesota,63,...,0.0,-2.0,1900-1959,126.0,107.0,81.0,1.0,596.0,3,NaN
4,1,9E,20363,13487,31650,MSP,"Minneapolis, MN",MN,Minnesota,63,...,0.0,-1.0,1900-1959,126.0,120.0,92.0,1.0,596.0,3,NaN


In [14]:
dataset.to_pickle('dataset.pkl')

# bike project

In [44]:
files=['201901','201902','201903','201904','201905','201906','201907','201908','201909', '201910']

def merge_data(trials_name):
    merged=pd.DataFrame()
    for name in trials_name:
        a1=pd.read_csv(name+'.csv', parse_dates=['Start date','End date'])
        a1=a1.groupby(pd.Grouper(key='Start date',freq='H')).agg({'Bike number':'count'}).reset_index()        
        print(name)
        print(a1.info())
        merged=merged.append(a1, ignore_index=True, sort=False)
        
    dr = pd.date_range(start='2019-01-01', end='2019-10-31')
    cal = calendar()
    holidays = cal.holidays(start=dr.min(), end=dr.max())
    # false is not a holiday, true is a holiday
    merged['Holiday'] = merged['Start date'].isin(holidays)
    merged['Holiday'] = (merged['Holiday']*1).astype('category')
    # Monday to Sunday is 0 to 6
    merged['weekday'] =merged['Start date'].dt.dayofweek.astype('category')
    #Winter month(1) is 12, 1,2, Spring(2) is 3,4,5,, summer(3) is 6,7,8, fall(4) is 9,10,11
    merged['Season']=((merged['Start date'].dt.month%12+3)//3).astype('category')
    return merged

In [45]:
dataset=merge_data(files)

201901
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 2 columns):
Start date     744 non-null datetime64[ns]
Bike number    744 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 11.7 KB
None
201902
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 2 columns):
Start date     672 non-null datetime64[ns]
Bike number    672 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 10.6 KB
None
201903
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 2 columns):
Start date     744 non-null datetime64[ns]
Bike number    744 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 11.7 KB
None
201904
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 2 columns):
Start date     720 non-null datetime64[ns]
Bike number    720 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 11.3 KB
N

In [46]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7296 entries, 0 to 7295
Data columns (total 5 columns):
Start date     7296 non-null datetime64[ns]
Bike number    7296 non-null int64
Holiday        7296 non-null category
weekday        7296 non-null category
Season         7296 non-null category
dtypes: category(3), datetime64[ns](1), int64(1)
memory usage: 136.1 KB


In [54]:
dataset['Hour']=dataset['Start date'].dt.hour #hour is from 0 to 23

dataset['Month']=dataset['Start date'].dt.month

dataset.drop('Start date',axis=1,inplace=True)

In [55]:
dataset.head(5)

,Bike number,Holiday,weekday,Season,Hour,Month
0,60,1,1,1,0,1
1,107,0,1,1,1,1
2,102,0,1,1,2,1
3,25,0,1,1,3,1
4,14,0,1,1,4,1


In [56]:
x_train,x_test,y_train,y_test=train_test_split(dataset.drop('Bike number',axis=1),dataset['Bike number'],test_size=0.25,random_state=42)

In [62]:
models=[RandomForestRegressor(),AdaBoostRegressor(),BaggingRegressor(),SVR(),KNeighborsRegressor(), LinearRegression()]
model_names=['RandomForestRegressor','AdaBoostRegressor','BaggingRegressor','SVR','KNeighborsRegressor', 'LinearRegression']
mse=[]
acc=[]
for i in range (len(models)):
    clf=models[i]
    clf.fit(x_train,y_train)
    test_pred=clf.predict(x_test)
    mse.append(mean_squared_error(test_pred,y_test))
    #acc.append(accuracy_score(test_pred, y_test))
rmsle_frame=pd.DataFrame({'Modelling Algo':model_names,'mse':mse})
print(rmsle_frame)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


          Modelling Algo            mse
0  RandomForestRegressor   24129.511737
1      AdaBoostRegressor   52277.638554
2       BaggingRegressor   24043.419677
3                    SVR  106023.796024
4    KNeighborsRegressor   22590.231623
5       LinearRegression  116463.371426


In [65]:
models=[LinearRegression(), Ridge(), Lasso()]
model_names=['LinearRegression', 'Ridge', 'Lasso']
mse=[]
for i in range(len(models)):
    clf=models[i]
    clf.fit(x_train, y_train)
    test_pred=clf.predict(x_test)
    mse.append(mean_squared_error(test_pred,y_test))
mse_frame=pd.DataFrame({'Modelling Algo':model_names,'mse':mse})
print(mse_frame)

     Modelling Algo            mse
0  LinearRegression  116463.371426
1             Ridge  116466.359030
2             Lasso  116476.484192


In [68]:
ridge_m = Ridge(random_state=42)
ridge_params = { 'max_iter':[3000],'alpha':[1, 10,100,200,300,400,800,900,1000]}
gs_ridge_m = GridSearchCV( ridge_m,ridge_params,scoring = 'neg_mean_squared_error',cv=5)
gs_ridge_m.fit( x_train, y_train )
test_pred = gs_ridge_m.predict(x_test)
print (gs_ridge_m.best_params_)
print ("mse: ", mean_squared_error(test_pred, y_test))

{'alpha': 400, 'max_iter': 3000}
mse:  116476.17390768466
